In [1]:
# Install required packages
!pip install folium groq

In [2]:
import os
import json
import folium
import groq
from dotenv import load_dotenv

In [3]:
# Load environment variables from .env file
load_dotenv()

# Set your Groq API key
GROQ_API_KEY = os.environ.get("GROQ_API_KEY", "")
if not GROQ_API_KEY:
    GROQ_API_KEY = input("Enter your Groq API key: ")
    os.environ["GROQ_API_KEY"] = GROQ_API_KEY

# Initialize Groq client
client = groq.Client(api_key=GROQ_API_KEY)

In [4]:
def process_query_with_groq(query):
    """Process the user's map query using Groq API and return the map configuration"""
    
    system_prompt = """
    You are a geographic visualization expert. Your task is to convert natural language requests about 
    geographic features into a JSON configuration that can be used to create an interactive map.
    
    Return a JSON object with the following structure:
    {
        "title": "Map title",
        "description": "Brief description of what the map shows",
        "center": [latitude, longitude],
        "zoom": zoom_level (1-20),
        "features": [
            {
                "type": "country, state, ocean, etc.",
                "name": "Feature name",
                "location": [latitude, longitude],
                "color": "hex color code"
            }
        ]
    }
    
    Respond with ONLY the JSON. No explanations or other text.
    """
    
    response = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Generate a map configuration for: {query}"}
        ],
        max_tokens=1024
    )
    
    # Extract and parse the JSON response
    try:
        response_content = response.choices[0].message.content.strip()
        # Handle cases where the response might include markdown code blocks
        if response_content.startswith("```json") and response_content.endswith("```"):
            response_content = response_content[7:-3].strip()
        elif response_content.startswith("```") and response_content.endswith("```"):
            response_content = response_content[3:-3].strip()
        
        map_config = json.loads(response_content)
        return map_config
    except Exception as e:
        print(f"Error parsing Groq response: {e}")
        print(f"Raw response: {response.choices[0].message.content}")
        return None

# Define valid folium colors
VALID_FOLIUM_COLORS = {
    "red", "blue", "green", "purple", "orange", "darkred", "lightred", "beige",
    "darkblue", "darkgreen", "cadetblue", "darkpurple", "white", "pink",
    "lightblue", "lightgreen", "gray", "black", "lightgray"
}

def create_map_from_config(map_config):
    """Create a Folium map using the configuration from Groq"""
    
    # Set the center and zoom level
    center = map_config.get("center", [0, 0])
    zoom = map_config.get("zoom", 2)
    
    # Create base map
    m = folium.Map(location=center, zoom_start=zoom, tiles="OpenStreetMap")
    
    # Add title as a caption
    title = map_config.get("title", "Generated Map")
    folium.map.Marker(
        center,
        icon=folium.DivIcon(
            icon_size=(500, 20),
            icon_anchor=(250, 20),
            html=f'<div style="font-size: 16pt; font-weight: bold; text-align: center;">{title}</div>'
        )
    ).add_to(m)
    
    # Process each feature
    features = map_config.get("features", [])
    for feature in features:
        name = feature.get("name", "Feature")
        location = feature.get("location", center)
        color = feature.get("color", "red")  # Default to red if not provided
        
        # Ensure color is valid
        color = color.lower()
        if color.startswith("#"):  # Convert hex to a valid folium color
            color = "blue"  # Default to blue (or map hex codes to a color dictionary)
        elif color not in VALID_FOLIUM_COLORS:
            color = "blue"  # Default if invalid
        
        # Add marker
        folium.Marker(
            location=location,
            popup=name,
            icon=folium.Icon(color=color, icon="info-sign")
        ).add_to(m)
    
    return m

In [5]:
# Example usage
def generate_map(query):
    print(f"Processing query: {query}")
    map_config = process_query_with_groq(query)
    if map_config:
        print("Creating map...")
        m = create_map_from_config(map_config)
        return m
    else:
        print("Failed to generate map configuration.")
        return None

In [6]:
# Test with an example
m = generate_map("where is Galle?")
if m:
    # Display map (in Jupyter)
    display(m)

Processing query: where is Galle?
Creating map...
